In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

In [3]:
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

In [5]:
#same thing with the functional interface
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [8]:
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 0s 949us/step - loss: 11.7792
Epoch 2/10
8/8 [==============================] - 0s 1ms/step - loss: 12.4481
Epoch 3/10
8/8 [==============================] - 0s 1ms/step - loss: 13.6903
Epoch 4/10
8/8 [==============================] - 0s 789us/step - loss: 15.6289
Epoch 5/10
8/8 [==============================] - 0s 928us/step - loss: 18.0628
Epoch 6/10
8/8 [==============================] - 0s 737us/step - loss: 20.9031
Epoch 7/10
8/8 [==============================] - 0s 828us/step - loss: 24.1852
Epoch 8/10
8/8 [==============================] - 0s 834us/step - loss: 27.4793
Epoch 9/10
8/8 [==============================] - 0s 900us/step - loss: 31.2976
Epoch 10/10
32/32 [==============================] - 0s 529us/step - loss: 36.2870


### Model with many inputs (some kind of a chatbot with natural language input and referrence text)

In [11]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None, ), 
                   dtype='int32', 
                   name='text')

embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,),
                      dtype='int32',
                      name='question')

embedded_question = layers.Embedding(64, question_vocabulary_size)(question_input)

encoded_question = layers.LSTM(32)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', 
             loss='categorical_crossentropy',
             metrics=['acc'])

In [14]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))

question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))


model.fit([text, question], answers, epochs=10, batch_size=128)

#or model.fit({'text' : text, 'question' : question}, answers, epochs=10, batch_size=128)

Epoch 1/10


InvalidArgumentError:  indices[24,4] = 9753 is not in [0, 64)
	 [[node model_2/embedding_2/embedding_lookup (defined at <ipython-input-12-d9fb78170147>:13) ]] [Op:__inference_train_function_12570]

Errors may have originated from an input operation.
Input Source operations connected to node model_2/embedding_2/embedding_lookup:
 model_2/embedding_2/embedding_lookup/10309 (defined at /Users/dawidkubicki/.conda/envs/Multi-Functional-Keras-API.git/lib/python3.8/contextlib.py:113)

Function call stack:
train_function
